In [390]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import json
import pandas as pd
import re

## 1/ Acquiring data

In [391]:
#Defining a request function to iterate more easily over pages.
def make_request(page_index, created_cookie):
    header2 = {"Accept":"application/json, text/javascript, */*; q=0.01", "Accept-Encoding":"gzip, deflate",
              "Accept-Language":"fr-FR,fr;q=0.8,en-US;q=0.6,en;q=0.4","Connection":"keep-alive",
              "Content-Type":"application/json; charset=UTF-8", "Host":"www.entreparticuliers.com",
              "Origin":"http://www.entreparticuliers.com", "Referer":"http://www.entreparticuliers.com/immobilier-location/",
              "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36",
              "X-Requested-With":"XMLHttpRequest","Cookie":created_cookie}
    payload2 = json.dumps({"pageIndex": page_index,"source":"undefined","latmin":"0","latmax":"0","lngmin":"0","lngmax":"0","frommoteur":"true"})
    response = requests.post("http://www.entreparticuliers.com/default.aspx/GetAnnonces", data = payload2, headers = header2)
    return json.loads(response.text)["d"]

In [392]:
#First cURL request designed to get the relevant cookie.
payload = json.dumps({'p_SearchParams': '{"rubrique":"1","ach_id":null,"FromMoteur":"true","lstSSTbien":"1","Caracteristique":17,"lstTbien":1,"meuble":"nc","OrigineAlerte":"SaveSearchMoteurHome","pays":"fra","prix_min":null,"prix_max":null,"lstThemes":"","lstNbPieces":0,"lstNbChambres":null,"surface_min":null,"surface_max":null,"CapaciteMin":null,"localisationType":4,"reference":"","rayon":0,"localisation_id_rayon":null,"lstLocalisationId":"121154","photos":0,"colocation":"","pageNumber":1,"order_by":1,"sort_order":1,"top":25,"SaveSearch":"false","EmailUser":"","GSMUser":""}', "forcealerte":'0'})
header = {'Referer': 'http://www.entreparticuliers.com/immobilier-location/', 'Accept': "application/json, text/javascript, */*; q=0.01", 'Accept-Encoding': "gzip, deflate", 'Accept-Language': "fr-FR,fr;q=0.8,en-US;q=0.6,en;q=0.4", 'Referer': "http://www.entreparticuliers.com/immobilier-location/", 'Origin': "http://www.entreparticuliers.com", 'Host': "www.entreparticuliers.com", 'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36", 'X-Request-With': "XMLHttpRequest", 'Content-Type': "application/json; charset=UTF-8"}
create_searchParams = requests.post("http://www.entreparticuliers.com/default.aspx/CreateSearchParams", data= payload, headers=header)
created_cookie = create_searchParams.headers["Set-Cookie"]

#Formatting features in a list of lists.
announces = []
page_index = 1
text = make_request(page_index, created_cookie)
while text:
    soup = BeautifulSoup(text, 'html.parser')
    announces +=[{items.name:items.string for items in annonce.children} for annonce in soup.arrayofannoncepresentation.find_all('annoncepresentation')]
    page_index += 1
    print("Page number: " + str(page_index))
    text = make_request(page_index, created_cookie)
    

Page number: 2
Page number: 3
Page number: 4
Page number: 5
Page number: 6
Page number: 7
Page number: 8
Page number: 9
Page number: 10
Page number: 11
Page number: 12
Page number: 13
Page number: 14
Page number: 15
Page number: 16


## 2/ Cleaning data

#### Some useful functions:

In [393]:
#Function separating number of rooms and surface
def split_infos(text):
    if text:
        if "/" in text:
            infos = re.split("/", text)
            infos = int(re.sub("pièce{1}s?", "", infos[0])), int(re.sub("m²", "", infos[1]))
        elif "m²" in text:
            infos = [None, int(re.sub("m²", "", text))]
        elif "pièces" in text:
            infos = [int(re.sub("pièce{1}s?", "", text)), None]
    else:
        infos = [None, None]
        
    return infos

#Function checking if the offer is also on leboncoin
def detect_leboncoin(link):
    return "leboncoin" in link

#### Cleaning process:

In [394]:
raw_data = pd.DataFrame(announces)
selected_data = raw_data[[col for col in raw_data.columns.values if col in ["liendetail", "prix", "localisation", "codepostal", 
                        "miniinfos", "lienimage1", "nbphotos", "source", "description", "idannonce", 
                         "latitude", "longitude", "titre"]]]  
selected_data["prix"] = selected_data["prix"].replace("&euro;|\s", "", regex = True)
selected_data[["nb_pieces", "surface"]] = pd.DataFrame(selected_data["miniinfos"].apply(split_infos, 0).tolist())
selected_data["leboncoin"] = selected_data["liendetail"].apply(detect_leboncoin, 0)
selected_data[["longitude", "latitude", "nbphotos", "prix"]] = selected_data[["longitude", "latitude", "nbphotos", "prix"]].apply(pd.to_numeric)
selected_data.to_csv("entreparticulier.csv")

/Users/gabrielducrocq/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/gabrielducrocq/anaconda/lib/python3.5/site-packages/pandas/core/frame.py:2378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/Users/gabrielducrocq/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta